In [1]:
import pandas as pd

In [2]:
df_binary = pd.read_csv('../emg_balanced_binary.csv', keep_default_na=False)

In [3]:
df_binary['gesture'].unique()

array(['index_finger', 'middle_finger', 'ring_finger', 'little_finger',
       'thumb', 'rest', 'victory_gesture', 'synthetic'], dtype=object)

In [4]:
df_binary['gesture'].value_counts()

gesture
synthetic          24036
little_finger       1084
ring_finger         1045
index_finger        1039
middle_finger       1017
thumb               1016
victory_gesture      895
rest                 726
Name: count, dtype: int64

In [12]:
# --- Importaciones necesarias ---
import pandas as pd

# --- Asumimos que 'df_binary' ya está cargado en memoria ---

# --- 1. Definir los parámetros de selección ---

# Número de filas deseadas por gesto antes de reducir a una única fila
n_filas_por_gesto = 700  # Puedes ajustar este valor

# Lista de gestos específicos a seleccionar (excluyendo 'synthetic')
gestos_a_seleccionar = [
    'index_finger',
    'middle_finger',
    'ring_finger',
    'little_finger',
    'thumb',
    'rest',
    'victory_gesture'
]

# --- 2. Crear DataFrames separados por 'gb_score' ---

# Filtrar filas con gb_score == 1 (positivo)
df_positivo = df_binary[df_binary['gb_score'] == 1]

# Filtrar filas con gb_score == 0 (negativo)
df_negativo = df_binary[df_binary['gb_score'] == 0]

# --- 3. Seleccionar una muestra de filas por gesto para el DataFrame positivo ---

# Inicializar lista para almacenar las muestras positivas
muestras_positivas = []

for gesto in gestos_a_seleccionar:
    df_gesto = df_positivo[df_positivo['gesture'] == gesto]
    disponible = len(df_gesto)

    if disponible >= n_filas_por_gesto:
        muestra_gesto = df_gesto.sample(n=n_filas_por_gesto, random_state=42)
    else:
        # Si no hay suficientes, tomar todas las disponibles
        muestra_gesto = df_gesto.copy()
    
    muestras_positivas.append(muestra_gesto)

muestra_positivo = pd.concat(muestras_positivas, ignore_index=True)

# --- 4. Seleccionar una muestra de filas por gesto para el DataFrame negativo ---

# Inicializar lista para almacenar las muestras negativas
muestras_negativas = []

for gesto in gestos_a_seleccionar:
    df_gesto = df_negativo[df_negativo['gesture'] == gesto]
    disponible = len(df_gesto)

    if disponible >= n_filas_por_gesto:
        muestra_gesto = df_gesto.sample(n=n_filas_por_gesto, random_state=42)
    else:
        muestra_gesto = df_gesto.copy()
    
    muestras_negativas.append(muestra_gesto)

muestra_negativo = pd.concat(muestras_negativas, ignore_index=True)

# --- 5. Dejar UNA ÚNICA FILA por gesto (selección aleatoria) ---

# Para muestra_positivo
muestra_positivo_unica = (
    muestra_positivo
    .groupby('gesture', group_keys=False)
    .apply(lambda x: x.sample(1, random_state=42), include_groups=False)  # <- Cambiado
    .reset_index(drop=True)
)

# Para muestra_negativo
muestra_negativo_unica = (
    muestra_negativo
    .groupby('gesture', group_keys=False)
    .apply(lambda x: x.sample(1, random_state=42), include_groups=False)  # <- Cambiado
    .reset_index(drop=True)
)

# --- 6. Guardar los DataFrames resultantes en archivos CSV ---

# Muestras completas (antes de reducción a una fila)
muestra_positivo.to_csv('muestra_positiva_gestos.csv', index=False)
muestra_negativo.to_csv('muestra_negativa_gestos.csv', index=False)

# Muestras reducidas (una fila por gesto)
muestra_positivo_unica.to_csv('muestra_positiva_gestos_UNICA.csv', index=False)
muestra_negativo_unica.to_csv('muestra_negativa_gestos_UNICA.csv', index=False)

# --- Fin del Script ---


In [1]:
import pandas as pd

In [2]:
muestra_pos = pd.read_csv('muestra_positiva_gestos_UNICA.csv', keep_default_na=False)
muestra_pos.columns

Index(['standard_deviation_e1', 'standard_deviation_e2',
       'standard_deviation_e3', 'standard_deviation_e4',
       'standard_deviation_e5', 'standard_deviation_e6',
       'standard_deviation_e7', 'standard_deviation_e8', 'root_mean_square_e1',
       'root_mean_square_e2', 'root_mean_square_e3', 'root_mean_square_e4',
       'root_mean_square_e5', 'root_mean_square_e6', 'root_mean_square_e7',
       'root_mean_square_e8', 'minimum_e1', 'minimum_e2', 'minimum_e3',
       'minimum_e4', 'minimum_e5', 'minimum_e6', 'minimum_e7', 'minimum_e8',
       'maximum_e1', 'maximum_e2', 'maximum_e3', 'maximum_e4', 'maximum_e5',
       'maximum_e6', 'maximum_e7', 'maximum_e8', 'zero_crossings_e1',
       'zero_crossings_e2', 'zero_crossings_e3', 'zero_crossings_e4',
       'zero_crossings_e5', 'zero_crossings_e6', 'zero_crossings_e7',
       'zero_crossings_e8', 'average_amplitude_change_e1',
       'average_amplitude_change_e2', 'average_amplitude_change_e3',
       'average_amplitude_change_

In [6]:
import pandas as pd

# Se asume que df_binary ya está cargado en memoria

# Gestos seleccionados
gestos_a_seleccionar = [
    'index_finger',
    'middle_finger',
    'ring_finger',
    'little_finger',
    'thumb',
    'rest',
    'victory_gesture'
]

# Separar en positivos y negativos
df_positivo = df_binary[df_binary['gb_score'] == 1]
df_negativo = df_binary[df_binary['gb_score'] == 0]

# Seleccionar una única fila por gesto en positivos
muestra_positivo_unica = df_positivo[df_positivo['gesture'].isin(gestos_a_seleccionar)] \
                         .groupby('gesture', group_keys=False).head(1)

# Seleccionar una única fila por gesto en negativos
muestra_negativo_unica = df_negativo[df_negativo['gesture'].isin(gestos_a_seleccionar)] \
                         .groupby('gesture', group_keys=False).head(1)

# Guardar muestras únicas
muestra_positivo_unica.to_csv("muestra_positiva_gestos_UNICA.csv", index=False)
muestra_negativo_unica.to_csv("muestra_negativa_gestos_UNICA.csv", index=False)

# Combinar muestras para excluirlas del dataset original
filas_a_excluir = pd.concat([muestra_positivo_unica, muestra_negativo_unica])

# Eliminar por índice del DataFrame original
emg_binary_final = df_binary.drop(index=filas_a_excluir.index)

# Guardar el dataset filtrado
emg_binary_final.to_csv("emg_binary_final.csv", index=False)
